In [ ]:
%pip install tensorflow
%pip install transformers
%pip install spacy

In [2]:
import pandas as pd
from flask import Flask, request, jsonify
from sklearn.model_selection import train_test_split
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [3]:
for encoding in ['utf-8', 'latin-1', 'windows-1252']:
    try:
        df = pd.read_csv('./sales_data_sample.csv', encoding=encoding)
        df['POSTALCODE'] = df['POSTALCODE'].fillna(0000)
        df[['QUANTITYORDERED', 'PRICEEACH', 'ORDERLINENUMBER', 'SALES']] = df[['QUANTITYORDERED', 'PRICEEACH', 'ORDERLINENUMBER', 'SALES']].fillna(0) 
        df[['STATUS', 'ADDRESSLINE1', 'ADDRESSLINE2', 'CITY', 'STATE',  "COUNTRY", "TERRITORY", "CONTACTLASTNAME","CONTACTFIRSTNAME", "DEALSIZE" ]] = df[['STATUS', 'ADDRESSLINE1', 'ADDRESSLINE2', 'CITY', 'STATE',  "COUNTRY", "TERRITORY", "CONTACTLASTNAME","CONTACTFIRSTNAME", "DEALSIZE" ]].fillna('N/A') 
        break 
    except UnicodeDecodeError:
        pass

In [ ]:
%pip install tf-keras


In [16]:
from transformers import AutoModelForTableQuestionAnswering, AutoTokenizer, pipeline
from transformers import TapasTokenizer

table = df.drop('ORDERDATE', axis=1)
table = table.astype(str)
table = table[:10]

model="google/tapas-large-finetuned-wtq"
tapas_model = AutoModelForTableQuestionAnswering.from_pretrained(model)
tapas_tokenizer = AutoTokenizer.from_pretrained(model)

nlp = pipeline('table-question-answering', model=tapas_model, tokenizer=tapas_tokenizer)

def qa(query,data):
    print(query)
    result = nlp({'table': data,'query':query})
    answer = result['cells']
    return answer[0]


In [18]:
app = Flask(__name__)

@app.route('/random_forest', methods=['POST'])
def recommend():
    user_ask = request.json
    question = user_ask['quest']
    prediction = qa(question, table)
    return jsonify({'answer': prediction})

if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


What was the total quantity ordered?


127.0.0.1 - - [02/Apr/2024 12:54:25] "POST /random_forest HTTP/1.1" 200 -
